In [102]:
import numpy as np

def parse_classification_results(path: str='./results/BlogCatalog.txt') -> tuple :
    (scores, pq) = [], []
    with open(path, 'r') as handle : 
        for line in handle.readlines():
            line = line.strip()
            pq.append([float(val[2:]) for val in line.split(':')[0][1:-1].split(',')])
            scores.append([float(col.strip().split()[-1]) for col in line.split('|')])
    (p, q) = zip(*pq)

    (p, q, scores) = map(
        np.array,
        zip(*sorted(zip(p, q, scores), key=lambda x: (x[0], x[1])))
    )

    (f1_base, gamma, f1_smooth) = scores.T

    return (p, q, f1_base, gamma, f1_smooth)

In [101]:
def summarize_classification_results(p, q, f1_base, gamma, f1_smooth) -> None :
    print(f'[mean] f1_base {f1_base.mean():.4f} | f1_smoothed {f1_smooth.mean():.4f}')
    print(f'[max-base] p {p[f1_base.argmax()]:.2f} | q {q[f1_base.argmax()]:.2f} | f1 {f1_base.max():.4f}')
    print(f'[max-smooth] p {p[f1_smooth.argmax()]:.2f} | q {q[f1_smooth.argmax()]:.2f} | gamma {gamma[f1_smooth.argmax()]:.3e} | | f1 {f1_smooth.max():.4f}')
    print(f'[(smooth > base) rate] {(f1_smooth > f1_base).mean():.4f}')
    if not((f1_smooth > f1_base).all()) :
        idx = np.argwhere(f1_smooth < f1_base).flatten()
        print('\tBase better than smoothed at:')
        for i in idx:
            print(f'\t[p={p[i]:.2f}; q={q[i]:.2f}] f1_base {f1_base[i]:.4f} | f1_smoothed {f1_smooth[i]:.4f}')
    print(f'[gain] {(f1_smooth.max() - f1_base.max())/f1_base.max()*100:.2f}%')

# Node Classification Task
### BlogCatalog

In [103]:
summarize_classification_results(*parse_classification_results('./results/BlogCatalog.txt'))

[mean] f1_base 0.9679 | f1_smoothed 0.9731
[max-base] p 4.00 | q 1.00 | f1 0.9735
[max-smooth] p 0.50 | q 4.00 | gamma 2.154e-07 | | f1 0.9764
[(smooth > base) rate] 0.9600
	Base better than smoothed at:
	[p=4.00; q=1.00] f1_base 0.9735 | f1_smoothed 0.9716
[gain] 0.30%


### PPI

In [104]:
summarize_classification_results(*parse_classification_results('./results/PPI.txt'))

[mean] f1_base 0.0713 | f1_smoothed 0.1096
[max-base] p 0.25 | q 1.00 | f1 0.0757
[max-smooth] p 4.00 | q 1.00 | gamma 4.642e-01 | | f1 0.1132
[(smooth > base) rate] 1.0000
[gain] 49.54%


### Wikipedia

In [105]:
summarize_classification_results(*parse_classification_results('./results/Wikipedia.txt'))

[mean] f1_base 0.0524 | f1_smoothed 0.1194
[max-base] p 4.00 | q 4.00 | f1 0.0545
[max-smooth] p 0.25 | q 0.50 | gamma 1.000e+00 | | f1 0.1258
[(smooth > base) rate] 1.0000
[gain] 130.83%
